In [3]:
import cv2
import numpy as np
import os
import face_recognition
from datetime import datetime

In [10]:
# from PIL import Image Grab
path = "Training_images"
images = []
classNames = []
myList = os.listdir(path)
# print(myList)

for cl in myList:
    curImg = cv2.imread(f"{path}/{cl}")
    images.append(curImg)
    #print(os.path.splitext(cl)[0])
    classNames.append(os.path.splitext(cl)[0])
print(classNames)

['Feranmi']


In [11]:
def findEncodings(images):
    encode_list = []

    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encode_list.append(encode)
    return encode_list

In [13]:
def mark_attendance(name):
    with open("Attendance.csv", "r+",) as f:
        my_data_list = f.readlines()

        name_list = []
        for line in my_data_list:
            entry = line.split(",")
            name_list.append(entry[0])
            if name not in name_list:
                now = datetime.now()
                date_string = now.strftime("%H:%M:%S")
                f.writelines(f"\n{name},{date_string}")

In [14]:
encode_list_known = findEncodings(images)
print("Encoding complete")

Encoding complete


In [ ]:
cap = cv2.VideoCapture(0)

while True:
    success, img = cap.read()
    imgS = cv2.resize(img, (0,0), None, 0.25, 0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)

    facesCurFrame = face_recognition.face_locations(imgS)
    encodesCurFrame = face_recognition.face_encodings(imgS, facesCurFrame)

    for encodeFace, faceLoc in zip(encodesCurFrame, facesCurFrame):
        matches = face_recognition.compare_faces(encode_list_known, encodeFace)
        faceDis = face_recognition.face_distance(encode_list_known, encodeFace)

        matchIndex = np.argmin(faceDis)

        if matches[matchIndex]:
            name = classNames[matchIndex].upper()
            y1, x2, y2, x1 = faceLoc
            y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
            cv2.putText(img, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            mark_attendance(name)

    cv2.imshow("Webcam", img)
    cv2.waitKey(1)